# TensorFlow 101

### 1. Basic syntax
### 2. TensorFlow Graphs
### 3. tf.Variable
### 4. tf.Placeholder
### 5. TensorFlow Neural Network
### 6. TensorFlow Regression and Classification
### 7. Estimator API
### 8. Saving and Restoring Model

## 1. Basic syntax

#### Version

In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

1.8.0


Tensor (Hello World)

In [3]:
hello = tf.constant("hello")

In [6]:
type(hello)

tensorflow.python.framework.ops.Tensor

In [8]:
world = tf.constant(" world")

#### Session:

In [10]:
with tf.Session() as sess:
    result = sess.run(hello + world)

In [11]:
print(result)

b'hello world'


#### matrix multiplication

In [12]:
a = tf.constant([[1,2],[3,4]])
a.get_shape()

TensorShape([Dimension(2), Dimension(2)])

In [13]:
b = tf.constant([ [1], [2] ])
b.get_shape()

TensorShape([Dimension(2), Dimension(1)])

In [14]:
# matmul
c = tf.matmul(a, b)

In [15]:
c

<tf.Tensor 'MatMul:0' shape=(2, 1) dtype=int32>

### Session

low level API

1. TensorFlow uses the tf.Session class to represent a connection between the client program
2. A tf.Session object provides access to devices in the local machine, and remote devices using the distributed TensorFlow runtime. 
3. It also caches information about your tf.Graph so that you can efficiently run the same computation multiple times.

In [16]:
with tf.Session() as sess:
    result = sess.run(c)

In [17]:
print(result)

[[ 5]
 [11]]


## 2. TensorFlow Graphs

Build Graph

Graphs are sets of connected nodes.  
In TensorFlow each node is an operation with possible inputs that can supply some output.

![caption](img/graph_example.png)

In [18]:
n1 = tf.constant(1)
n2 = tf.constant(2)

In [19]:
n3 = n1 + n2

In [20]:
with tf.Session() as sess:
    result = sess.run(n3)

In [21]:
print(result)

3


In [22]:
print(n3)

Tensor("add_2:0", shape=(), dtype=int32)


default graph

In [24]:
default = tf.get_default_graph()

In [26]:
print(default)

In [28]:
g = tf.Graph()

In [30]:
print(g)

In [32]:
with g.as_default():
    print(g is tf.get_default_graph())

True


In [33]:
print(g is tf.get_default_graph())

False


## 3. Variables

* During the optimization process, TensorFlow tunes the parameters of the model
* Variables can hold the values of weights and biases throughout the session
* Variables need to be initialized


What is the lifetime of a variable?

A variable is created when you first run the tf.Variable.initializer operation for that variable in a session. It is destroyed when that tf.Session.close.

In [35]:
my_tensor = tf.random_uniform((4, 4), 0, 1)

In [36]:
my_var = tf.Variable(initial_value=my_tensor)

In [37]:
print(my_var)

<tf.Variable 'Variable:0' shape=(4, 4) dtype=float32_ref>


In [38]:
init = tf.global_variables_initializer()

In [39]:
with tf.Session() as sess:
    sess.run(init)
    result = sess.run(my_var)

In [40]:
print(result)

[[0.4482218  0.94580424 0.9219643  0.04163468]
 [0.28762555 0.29112446 0.06869078 0.1168139 ]
 [0.92229295 0.8543055  0.6982882  0.6842557 ]
 [0.2689718  0.87218726 0.5694454  0.15720558]]


## 4. Placeholders

Placeholders are initially empty and are used to feed in the actual training examples.  
However they do need a declared expected data type (tf.float32) with a optional shape argument

In [42]:
ph = tf.placeholder(tf.float32)

In [43]:
ph = tf.placeholder(tf.float32, shape=(None, 5))

## 5. TensorFlow Neural Network

1. Build a graph.
2. Initiate the Session.
3. Feed data in and get output.

![caption](img/snn1.png)

In [44]:
import numpy as np
import tensorflow as tf
np.random.seed(99)
tf.set_random_seed(99)

## Feeddict

data

In [45]:
rand_a = np.random.uniform(0,100,(5,5))
rand_a

array([[67.22785586, 48.80783992, 82.5495174 ,  3.14463876, 80.80499634],
       [56.56174196, 29.76224987,  4.66957205, 99.06273995,  0.6825733 ],
       [76.97930282, 74.6767101 , 37.74389363, 49.41474522, 92.89483921],
       [39.54540443, 97.39562968, 52.44147154,  9.36130933, 81.33084128],
       [21.16867856, 55.43457846, 29.2269116 , 81.614236  , 82.80425662]])

In [46]:
rand_b = np.random.uniform(0,100,(5,1))
rand_b

array([[22.15773717],
       [64.48347016],
       [ 9.51816219],
       [41.16632388],
       [ 9.68652613]])

placeholder

In [47]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

add_op and mul_op

In [49]:
add_op = a + b
mul_op = a * b

#### Running Sessions  to create Graphs with Feed Dictionaries

In [51]:
with tf.Session() as sess:
    add_result = sess.run(add_op, feed_dict={a: rand_a, b: rand_b})


In [52]:
print(add_result)

[[ 89.3856    70.965576 104.70725   25.302376 102.96273 ]
 [121.04521   94.24571   69.15304  163.5462    65.16604 ]
 [ 86.49747   84.19487   47.262054  58.932907 102.412994]
 [ 80.71173  138.56195   93.607796  50.527634 122.49716 ]
 [ 30.855206  65.1211    38.913437  91.30076   92.49078 ]]


#### Example Neural Network

In [53]:
n_features = 10
n_dense_neurons = 3

In [54]:
# Placeholder for x
x = tf.placeholder(tf.float32,(None,n_features))

In [55]:
# Variables for w and b
b = tf.Variable(tf.zeros([n_dense_neurons]))

W = tf.Variable(tf.random_normal([n_features,n_dense_neurons]))

Activation Function

In [56]:
xW = tf.matmul(x, W)

In [57]:
z = xW + b

In [58]:
a = tf.sigmoid(z)

Variables init

In [59]:
init = tf.global_variables_initializer()

In [60]:
with tf.Session() as sess:
    sess.run(init)
    layout = sess.run(a, feed_dict={x: np.random.random([1, n_features])})

In [61]:
print(layout)

[[0.8665317  0.4787856  0.60856485]]
